Suppose $A \in \mathbb{R}^{n \times m}, B \in \mathbb{R}^{m \times r}$ then the matrix product $AB := A \cdot B \in \mathbb{R}^{n \times r}$ is defined by
$$
(A \cdot B)_{ij} = \sum_{k = 1}^m A_{ik} \cdot B_{kj}
$$
for $1 \leq i \leq n, 1 \leq j \leq r$.

In [23]:
import numpy as np

A = np.array([
    [-1, 1, 2, 3],
    [-3, 4, 5, 6],
    [0, 7, 8, 9]
])
print(f"{A.shape=}")
n, m = A.shape
B = np.array([
    [-4, 5],
    [8, 1],
    [-1, 0],
    [6, -2]
])
print(f"{B.shape=}")
m2, k = B.shape
C = A @ B
print(f"C = \n{C}")
print(f"{C.shape=}")
assert m == m2
assert C.shape == (n, k)

A.shape=(3, 4)
B.shape=(4, 2)
C = 
[[ 28 -10]
 [ 75 -23]
 [102 -11]]
C.shape=(3, 2)


We denote by 
$$
A_{i, -} := \begin{pmatrix}A_{i, 1}, A_{i, 2}, \dots, A_{i, n}\end{pmatrix} \in \R^{1 \times N} \cong \R^N
$$
and
$$
B_{-, j} := \begin{pmatrix}B_{i, 1} \\ B_{i, 2} \\ \vdots \\ B_{i, n}\end{pmatrix} \in \R^{N \times 1} \cong \R^N
$$
the $i$-th row and $j$-th column of $A$ and $B$ respectively.

The dot product between two vectors $v = (v_1, \dots, v_n), w = (w_1, \dots, w_n) \in \mathbb{R}^n$ is defined by
$$
v \cdot w := \sum­_{i = 1}^N v_i w_i \in \mathbb{R}.
$$

With this notation we can then write
$$
(AB)_{ij} = A_{i, -} \cdot B_{-, j}.
$$

In [26]:
A_2nd_row = A[1, :]
B_1st_col = B[:, 0]

print(f"{A_2nd_row=}")
print(f"{B_1st_col=}")

print(f"{C[1, 0]=}")
assert C[1, 0] == np.dot(A_2nd_row, B_1st_col)

A_2nd_row=array([-3,  4,  5,  6])
B_1st_col=array([-4,  8, -1,  6])
C[1, 0]=75


As such, if we apply the naive algorithm (i.e. just implement the definition directly) we will need to compute a dot product between a row and a column of $A$ and $B$ respectively. Those have lenght $m$ each so to compute the dot product we need $O(m)$ operations. And this has to be done for each of the $n \times r$ entries in $AB$, meaning that the total number of operations is $O(nmr)$.

For simplicities' sake we will only restrict ourselves on the case of multiplying square matrices, i.e., $n = m = r$. We write $N := n$, and with that we have
$$
A, B, AB \in \mathbb{R}^N.
$$

To leverage Cache locality more effectively we will split the matrices into blocks of fixed size. Let $\beta \in \N$ be a fixed number that we call the block size. This is usually written as a multiple of $2$ and we wil be using $\beta := 32$.

To avoid having to deal with partial blocks we assume that $N = k \beta$ for some $k \in \N$. With that we can write
$$
A, B, AB \in \mathbb{R^{k\beta \times k\beta}}
$$
and for $1 \leq i, j \leq n$ we define the i,j-th block of $A$ by
$$
(A^\square_{ij})_{a, b} := A_{i \beta + a, j \beta + b}
$$
for $1 \leq a, b, \beta$ where $A^\square \in \R^{\beta \times \beta}$. This representation immediately comes from the fact that $R^{k \beta, k\beta}$ is isomorphic to $(R^{\beta \times \beta})^{k \times k}$.

With this block representation we can then write
$$
C^\square_{ij} = \sum­_{r = 1}^k A^\square_{i, k} \cdot B^\square_{k, j}
$$
where $A^\square_{i, k} \cdot B^\square_{k, j}$ denotes the matrix multiplication of the blocks and the addition is taken element-wise.

In [68]:
import numpy as np
BLOCK_SIZE = 2 # beta
A = np.array([
    [1, 2, -1, 4],
    [-1, 5, 0, -4],
    [5, -2, 1, 0],
    [0, 0, 5, -2]
])
A_blocks = [
    [A[0:BLOCK_SIZE, 0:BLOCK_SIZE], A[0:BLOCK_SIZE, BLOCK_SIZE:2 * BLOCK_SIZE]],
    [A[BLOCK_SIZE:2*BLOCK_SIZE, 0:BLOCK_SIZE], A[BLOCK_SIZE:2*BLOCK_SIZE, BLOCK_SIZE:2 * BLOCK_SIZE]],
]
print("A_blocks")
print(A_blocks[0][0])
print(A_blocks[0][1])
print(A_blocks[1][0])
print(A_blocks[1][1])
print()
B = np.array([
    [-4, 5, 5, 1],
    [8, 1, -5, -1],
    [-1, 0, 3, 0],
    [6, -2, -1, 2],
])
B_blocks = [
    [B[0:BLOCK_SIZE, 0:BLOCK_SIZE], B[0:BLOCK_SIZE, BLOCK_SIZE:2 * BLOCK_SIZE]],
    [B[BLOCK_SIZE:2*BLOCK_SIZE, 0:BLOCK_SIZE], B[BLOCK_SIZE:2*BLOCK_SIZE, BLOCK_SIZE:2 * BLOCK_SIZE]],
]
print("B_blocks")
print(B_blocks[0][0])
print(B_blocks[0][1])
print(B_blocks[1][0])
print(B_blocks[1][1])

C = A @ B
print()
print("C=")
print(C)
assert (C[0:BLOCK_SIZE, 0:BLOCK_SIZE] == A[0:BLOCK_SIZE, :] @ B[:, 0:BLOCK_SIZE]).all()

C_topleft = C[0:BLOCK_SIZE, 0:BLOCK_SIZE]

prod_1 = A_blocks[0][0] @ B_blocks[0][0]
prod_2 = A_blocks[0][1] @ B_blocks[1][0] 
print(f"{prod_1=}")
print(f"{prod_2=}")

print("Top left block in C:")
print(C_topleft)

assert (C_topleft == A_blocks[0][0] @ B_blocks[0][0] + A_blocks[0][1] @ B_blocks[1][0]).all()

A_blocks
[[ 1  2]
 [-1  5]]
[[-1  4]
 [ 0 -4]]
[[ 5 -2]
 [ 0  0]]
[[ 1  0]
 [ 5 -2]]

B_blocks
[[-4  5]
 [ 8  1]]
[[ 5  1]
 [-5 -1]]
[[-1  0]
 [ 6 -2]]
[[ 3  0]
 [-1  2]]

C=
[[ 37  -1 -12   7]
 [ 20   8 -26 -14]
 [-37  23  38   7]
 [-17   4  17  -4]]
prod_1=array([[12,  7],
       [44,  0]])
prod_2=array([[ 25,  -8],
       [-24,   8]])
Top left block in C:
[[37 -1]
 [20  8]]
